In [1]:
import pandas as pd

In [32]:
movie_metadata_header = [
    'ID',
    'Freebase movie ID',
    'Movie name',
    'Movie release date',
    'Movie box office revenue',
    'Movie runtime',
    'Movie languages',
    'Movie countries',
    'Movie genres'
]

character_metadata_header = [
    'ID',
    'Freebase movie ID',
    'Movie release date',
    'Character Name',
    'Actor DOB',
    'Actor gender',
    'Actor height',
    'Actor ethnicity',
    'Actor Name',
    'Actor age at movie release',
    'Freebase character map',
    'Freebase character ID',
    'Freebase actor ID'
]

In [125]:
# read oscar dataset
oscar_dataset = pd.read_csv('the_oscar_award.csv')
oscar_dataset = oscar_dataset.rename(columns = {'name': 'Actor Name','film' : 'Movie name','year_film':'Movie release year'})
# only take actor/actress Awards
oscar_nominees = oscar_dataset[oscar_dataset['category'].str.contains('ACTOR') | oscar_dataset['category'].str.contains('ACTRESS')].reset_index(drop=True)
oscar_winners = oscar_nominees[oscar_nominees['winner'] == True]
actor_first_win = oscar_winners.groupby('Actor Name').agg({'Movie release year': 'min',
                                                           'year_ceremony'     : 'min'}).reset_index()
actor_first_nomination =  oscar_nominees.groupby('Actor Name').agg({'Movie release year': 'min',
                                                                    'year_ceremony'     : 'min',
                                                                    'ceremony'          : 'count',
                                                                    'winner'            : 'any'}).reset_index()
actor_first_win = actor_first_win.rename(columns = {'year_ceremony' :'first win date'}).drop(columns=['Movie release year'])
actor_first_nomination = actor_first_nomination.rename(columns = {'year_ceremony' :'first nomination date'}).drop(columns=['Movie release year'])
actor_first_nomination

,Actor Name,first nomination date,ceremony,winner
0,Abigail Breslin,2007,1,False
1,Adam Driver,2019,2,False
2,Adolph Caesar,1985,1,False
3,Adolphe Menjou,1931,1,False
4,Adriana Barraza,2007,1,False
...,...,...,...,...
930,Winona Ryder,1994,2,False
931,Woody Allen,1978,1,False
932,Woody Harrelson,1997,3,False
933,Yalitza Aparicio,2019,1,False


In [132]:
character_metadata = pd.read_csv(
    'MovieSummaries/character.metadata.tsv',
    delimiter='\t',
    names=character_metadata_header
)
# in order to compare release years
character_metadata['Movie release year'] = pd.to_numeric(character_metadata['Movie release date'].str.split('-').str[0],errors = 'coerce')
actors_in_movies = character_metadata[['ID','Character Name','Actor Name','Movie release year']]

In [140]:
oscars_in_movies = pd.merge(actors_in_movies,actor_first_nomination,on = 'Actor Name',how = 'left')
oscars_in_movies.fillna({'first nomination date': 9999,'ceremony':0, 'winner':False},inplace= True)
oscars_in_movies

,ID,Character Name,Actor Name,Movie release year,first nomination date,ceremony,winner
0,975900,Akooshay,Wanda De Jesus,2001.0,9999.0,0.0,False
1,975900,Lieutenant Melanie Ballard,Natasha Henstridge,2001.0,9999.0,0.0,False
2,975900,Desolation Williams,Ice Cube,2001.0,9999.0,0.0,False
3,975900,Sgt Jericho Butler,Jason Statham,2001.0,9999.0,0.0,False
4,975900,Bashira Kincaid,Clea DuVall,2001.0,9999.0,0.0,False
...,...,...,...,...,...,...,...
450664,913762,Elensh,Dorothy Elias-Fahn,1992.0,9999.0,0.0,False
450665,913762,Hibiki,Jonathan Fahn,1992.0,9999.0,0.0,False
450666,28308153,NaN,David Hemmings,1957.0,9999.0,0.0,False
450667,28308153,NaN,Roberta Paterson,1957.0,9999.0,0.0,False


In [63]:
movie_metadata = pd.read_csv(
    'MovieSummaries/movie.metadata.tsv',
    delimiter='\t',
    names=movie_metadata_header
)
# in order to compare release years
movie_metadata['Movie release year'] = pd.to_numeric(movie_metadata['Movie release date'].str.split('-').str[0],errors = 'coerce')
